In [2]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.6.0+cu124


In [3]:
# Install torch geometric
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
  !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
  !pip install torch-geometric
  !pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 29.5 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 831.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.

In [4]:
!pip install pynauty

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pynauty: filename=pynauty-2.8.8.1-cp311-cp311-linux_x86_64.whl size=181785 sha256=ea3c072552fda1caddf3080affc93ad441dcd242f490e7367e1d8d327836da42
  Stored in directory: /root/.cache/pip/wheels/c1/e3/ba/722f2d9187889ac73ce949422b5b698450d8d1390b479adab8
Successfully built pynauty


In [11]:
import networkx as nx
from pynauty import Graph, autgrp
from sklearn.model_selection import train_test_split
import random
import torch
from torch_geometric.data import Data
from sympy.combinatorics import Permutation, PermutationGroup


MAX_EXAMPLES_NUM = 30
MAX_ATTEMPTS = 100


def read_graphs_from_g6(file_path: str) -> list[Graph]:
    """
    Reads graphs from a .g6 file and converts them to igraph format.

    :param file_path: Path to the .g6 file.
    :returns: List of igraph graphs.
    """

    graphs = nx.read_graph6(file_path)
    pynauty_graphs = []
    for g in graphs:
        n = int(g.number_of_nodes())
        new_g = Graph(n)
        new_g.set_adjacency_dict(dict(g.adjacency()))
        pynauty_graphs.append((new_g, n, g.edges()))
    return pynauty_graphs


def generate_partial_automorphism_graphs(graphs: list[Graph]) -> list:
    """
    Generates partial automorphism graphs from a list of igraph graphs.

    :param graphs: List of igraph graphs.
    :returns: TODO
    """

    dataset = []

    for G, n, edge_list in graphs:
        gens_raw, group_size, _, _, _ = autgrp(G)

        # ensure 10-30 examples per graph with 1:1 ratio of positive to negative examples
        examples_num = int(min(MAX_EXAMPLES_NUM, group_size))
        gens = [Permutation(g) for g in gens_raw]
        group = PermutationGroup(gens)

        # positive examples
        positives = []
        seen = set()
        for _ in range(examples_num):
            perm = group.random().array_form
            k = random.randint(3, min(6, n))
            domain = random.sample(range(n), k)
            mapping = {i: perm[i] for i in domain}
            key = frozenset(mapping.items())
            if key in seen:
                continue
            seen.add(key)
            positives.append(mapping)
            dataset.append(_make_data(edge_list, n, mapping, 1))

        # negative examples
        for mapping in positives:
            u = random.choice(list(mapping.keys()))
            v_old = mapping[u]
            v_new = random.choice([v for v in range(n) if v != v_old])
            neg_map = mapping.copy()
            neg_map[u] = v_new
            key = frozenset(neg_map.items())
            if key in seen:
                continue
            seen.add(key)
            dataset.append(_make_data(edge_list, n, neg_map, label=0))

    return dataset


def _make_data(edge_list: list[tuple], n: int,  mapping: dict[int, int], label: int) -> Data:
    x = torch.zeros((n, 2), dtype=torch.float)
    map_tensor = torch.full((n,), -1, dtype=torch.long)

    for i, j in mapping.items():
        x[i, 0] = 1.0
        x[j, 1] = 1.0
        map_tensor[i] = j

    if len(edge_list) > 0:
        edges = []
        for u, v in edge_list:
            edges.append([u, v])
            edges.append([v, u])
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)

    y = torch.tensor([label], dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, y=y, mapping=map_tensor)

    return data


raw_graphs = read_graphs_from_g6("2000_raw_graphs.g6")

graphs_train, graphs_val = train_test_split(raw_graphs, test_size=0.2, random_state=42)

train_dataset = generate_partial_automorphism_graphs(graphs_train)
val_dataset   = generate_partial_automorphism_graphs(graphs_val)

In [12]:
import torch
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_mean_pool


train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

class GIN(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=3):
        super().__init__()

        self.convs = nn.ModuleList()

        self.convs.append(
            GINConv(nn.Sequential(
                nn.Linear(2, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
            ))
        )

        for _ in range(num_layers - 1):
            self.convs.append(
                GINConv(nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, hidden_dim),
                ))
            )

        self.classifier = nn.Linear(hidden_dim, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.classifier(x).view(-1)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()


def train_epoch():
    model.train()
    total_loss = 0
    correct, total = 0, 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
        pred = (torch.sigmoid(out) > 0.5).float()
        correct += (pred == data.y).sum().item()
        total += data.num_graphs
    return total_loss / total, correct / total

@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total_correct = 0
    total = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = (out > 0).float()
        total_correct += (pred == data.y).sum().item()
        total += data.num_graphs
    return total_correct / total


for epoch in range(1, 101):
    train_loss, train_acc = train_epoch()
    val_acc = eval_epoch(val_loader)
    print(f"Epoch {epoch:02d} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.4f} | "
          f"Val Acc:   {val_acc:.4f}")


KeyboardInterrupt: 

In [8]:
!python -u GIN_full_aut.py

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZN2at4_ops16div__Tensor_mode4callERNS_6TensorERKS2_St8optionalIN3c1017basic_string_viewIcEEE
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_spmm_cuda.so: undefined symbol: _ZN5torch8autograd12VariableInfoC1ERKN2at6TensorE
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 01 | T